In [1]:
pip install moviepy==1.0.3

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from moviepy.editor import VideoFileClip

In [4]:
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
import os
import random

# Input folder names (each folder contains files)
folders = ["s2", "s3", "s12", "s13", "s28", "s29", "s30"]

# Paths and output files
output_audio_dir = "output_samples"  # Directory to save generated audio files
output_file = "output1.txt"  # Log file for generated audio files and tags
os.makedirs(output_audio_dir, exist_ok=True)

# Function to list all files in the specified folders
def list_files_in_folders(folders, extension=".mpg"):
    all_files = []
    for folder in folders:
        folder_path = os.path.abspath(folder)  # Get the full path of the folder
        if os.path.exists(folder_path):
            files = [
                (folder, os.path.join(folder_path, f))  # Store folder name and full path
                for f in os.listdir(folder_path)
                if f.endswith(extension)
            ]
            all_files.extend(files)
        else:
            print(f"Warning: Folder not found - {folder_path}")
    return all_files

# Function to load audio from a video file
def load_audio(file_path):
    video = VideoFileClip(file_path)  # Load the video file
    audio = video.audio
    audio_path = file_path.replace(".mpg", ".wav")  # Temporary WAV file to save the audio
    audio.write_audiofile(audio_path, logger=None)  # Extract audio to WAV format
    audio_segment = AudioSegment.from_file(audio_path, format="wav")  # Load as AudioSegment
    os.remove(audio_path)  # Delete the temporary WAV file
    return audio_segment

# List all files in the specified folders
all_files = list_files_in_folders(folders)

# Ensure there are enough files to sample from
if len(all_files) < 2:
    raise ValueError("Not enough files found to generate samples.")

# Open the output file for logging
with open(output_file, "w") as f:
    f.write("Output File, Speaker1, Speaker2, Tag\n")  # Header
    
    # Process for generating samples
    for idx in range(3000):
        # Randomly choose two files (X and Y) from the list of files
        (speaker1, file1_path), (speaker2, file2_path) = random.sample(all_files, 2)

        # Load audio for the selected files
        audio1 = load_audio(file1_path)
        audio2 = load_audio(file2_path)

        # Generate unique filename before using it
        output_filename = f"audio_{idx}.wav"

        # Randomly decide if the output is overlapped or non-overlapped
        is_overlapped = random.choice([True, False])

        if is_overlapped:
            # Adjust lengths and overlay the audio
            max_len = max(len(audio1), len(audio2))
            audio1 = audio1 + AudioSegment.silent(duration=max_len - len(audio1))
            audio2 = audio2 + AudioSegment.silent(duration=max_len - len(audio2))
            output_audio = audio1.overlay(audio2)
            tag = "overlapped"
            log_speaker1, log_speaker2 = speaker1, speaker2  # Store both speaker names
        else:
            # Choose one file randomly for non-overlapped
            chosen_speaker, chosen_audio = random.choice([(speaker1, audio1), (speaker2, audio2)])
            output_audio = chosen_audio
            tag = "non-overlapped"
            log_speaker1, log_speaker2 = (chosen_speaker, "") if chosen_speaker == speaker1 else ("", chosen_speaker)

        # Update filename with tag information
        output_filename = f"audio_{idx}_{tag}.wav"
        output_filepath = os.path.join(output_audio_dir, output_filename)
        output_audio.export(output_filepath, format="wav")

        # Log the result
        f.write(f"{output_filename},  {log_speaker1},  {log_speaker2},  {tag}\n")

print(f"Process completed. 3000 samples are saved in '{output_audio_dir}', and log is saved in '{output_file}'.")


Process completed. 3000 samples are saved in 'output_samples', and log is saved in 'output1.txt'.
